# Step 02 - Retrieve MLflow model from registry, build serving image, and push to target registry

Notebook is built to retrieve and download a MLflow model from a AML registry (however, this could be from any registry), then to create a serving docker image based on the included `Dockerfile` here. The dependencies contained within the MLflow's conda YAML file are installed into that image before it is pushed into a container registry.

### Import required packages and create connection to AML workspace

In [ ]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment, ModelPackage, BaseEnvironment, AzureMLOnlineInferencingServer, CodeConfiguration, DataCollector, DeploymentCollection
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import mlflow
import mlflow.sklearn

subscription_id = "..."
resource_group = "..."
workspace = "..."
model_name = "zone1-power-consumption-xgboost-model"

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)
ml_client

### Retrieve model and download a local copy

In [22]:
registered_model = ml_client.models.get(model_name, label='latest')

In [ ]:
import shutil
try:
    shutil.rmtree('./model')
except Exception as e:
    pass

# mlflow.sklearn.load_model(registered_model.path)
model_uri = f"runs:/{registered_model.job_name}/model"

model = mlflow.sklearn.load_model(model_uri)
mlflow.xgboost.save_model(model, './model')

### Build a custom serving container using docker python package

In [ ]:
import docker
from IPython.display import display, Markdown

def build_docker_image(dockerfile_path, image_name, context_path='.'):
    # Initialize the Docker client
    client = docker.from_env()

    # Build the Docker image
    try:
        image, logs = client.images.build(path=context_path, dockerfile=dockerfile_path, tag=image_name, rm=True)
        # Display logs
        for log in logs:
            if 'stream' in log:
                display(Markdown(f"```\n{log['stream']}\n```"))
        
        display(Markdown(f"**Docker image '{image_name}' built successfully.**"))
        return image

    except docker.errors.BuildError as e:
        display(Markdown(f"**Error occurred while building the Docker image: {e}**"))
    except docker.errors.APIError as e:
        display(Markdown(f"**Error communicating with Docker API: {e}**"))

# Specify paths
dockerfile_path = './Dockerfile'  # Path to your Dockerfile
image_name = 'zone-power-consumption:2'    # Name of the Docker image
context_path = '.'                # Context path for the build (where your Dockerfile and model directory are located)

# Build the Docker image
image = build_docker_image(dockerfile_path, image_name, context_path)


### Connect to container registry and push image

Azure Container Registry is targeted here, however by modifying docker login server and credentials images can be pushed to different directories

In [30]:
ml_client.workspaces.get(ml_client.workspace_name).container_registry

acr_name = ml_client.workspaces.get(ml_client.workspace_name).container_registry.split('/')[-1]
acr_name

acr_login_server = f'{acr_name}.azurecr.io'  # ACR login server

In [31]:
import os
os.environ['username'] = '...'
os.environ['password'] = '...'
username = os.environ['username']
password = os.environ['password']

In [32]:
! az acr login --name {acr_login_server} --username {username} --password {password}

The login server endpoint suffix '.azurecr.io' is automatically omitted.
Login Succeeded
WARNING! Your password will be stored unencrypted in /home/azureuser/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store


In [ ]:
def push_docker_image_to_acr(image, acr_name, image_name, acr_login_server):
    client = docker.from_env()

    # Tag the image
    acr_image_name = f"{acr_login_server}/{image_name}"
    image.tag(acr_image_name)
    
    # Push the image
    try:
        for line in client.images.push(acr_image_name, stream=True, decode=True):
            if 'status' in line:
                display(Markdown(f"```\n{line['status']}\n```"))

        display(Markdown(f"**Docker image '{acr_image_name}' pushed successfully to ACR.**"))

    except docker.errors.APIError as e:
        display(Markdown(f"**Error pushing Docker image to ACR: {e}**"))
        
push_docker_image_to_acr(image, acr_name, image_name, acr_login_server)